In [1]:
import numpy as np
import matplotlib.pylab as plt

# Heat penetration in soil

The temperature distribution in the semi-infinite region $x>0$ with its surface $x=0$ at the periodic temperature $v(t, x=0) = V_0\,sin(wt)$ is given by:

$$
v(x, t) = V_0 \, exp\left( -x \sqrt{\frac{w}{2 \kappa}} \right) \, sin\left( wt - x \sqrt{\frac{w}{2 \kappa}} \right)
$$

with:
* Thermal diffusivity: $\kappa = \frac{k}{\rho Cp}$ in $m^2/s$
* Thermal conduction: $k$  (or $\lambda$), in $W/m/K$
* Density: $\rho$ in $kg/m^3$
* Specific heat (chaleur volumic): $C_p$  in $J/kg/K$

See *Conduction of Heat in Solids*, H.S. Carslaw, page 319.

The penetration depth or characteristic distance is
$$
d_c = \sqrt{ \frac{2 \kappa}{w} }
$$

Let's evaluate this distance for periodic daily and annual temperature oscillations and common materials.

In [2]:
# Material properties:
materials = {}
materials['Concrete']  = {'k':1.8, 'rho':2300, 'Cp':1000}
materials['Wood wool'] = {'k':0.04, 'rho':160, 'Cp':2100}
materials['PSE'] = {'k':0.04, 'rho':34, 'Cp':145}
materials['Clay'] = {'k':1.28, 'rho':880, 'Cp':1450}

# Compute the thermal diffusivity : 
for props in materials.values():
    if 'kappa' not in props:
        props['kappa'] = props['k']/props['rho']/props['Cp']

In [3]:
# Angular frequency (rad/s)
w_day = 2*np.pi / (24 * 60 * 60)
w_year = 2*np.pi / (24 * 60 * 60 * 365)

In [4]:
def penetration_depth(kappa):
    delta_day = np.sqrt( 2*kappa/w_day )
    delta_year = np.sqrt( 2*kappa/w_year )
    
    return delta_year, delta_day

In [10]:
print('penetration depth:  annual & daily  in meter')
for name, props in materials.items():
    line = '{:>18}  {:.2f}   {:.2f}'.format(name,
                                            *penetration_depth(props['kappa']))
    print(line)

penetration depth:  annual & daily  in meter
          Concrete  2.80   0.15
               PSE  9.02   0.47
         Wood wool  1.09   0.06
              Clay  3.17   0.17


## Temperature distribution: Animation

using normed quantities

In [6]:
import matplotlib.animation as animation
from IPython.display import HTML

In [7]:
def T_soil_theo(u, t):
    return np.exp( -u ) * np.sin( 2*np.pi*t - u )

In [9]:
n_frames = 20
u = np.linspace(0, 5, 71)

fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111, autoscale_on=False, xlim=(0, 5), ylim=(-1, 1))

ax.plot(u, np.exp(-u), '--', color='red', linewidth=1)
ax.plot(u, -np.exp(-u), '--', color='red', linewidth=1)
ax.set_xlabel('x / critical depth');
ax.set_ylabel('T / T0');

line, = ax.plot([], [], '-', lw=2)
time_template = 'time = %.2f 2pi/w'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


def init():
    line.set_data(u, T_soil_theo(u, 0))
    time_text.set_text('hello')
    return line, time_text

def animate(i):
    thisx = u
    thisy = T_soil_theo(u, 1/n_frames*i)

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i/n_frames))
    return line, time_text

ani = animation.FuncAnimation(fig, animate, np.arange(1, n_frames),
                              interval=80, blit=False, init_func=init)

HTML(ani.to_jshtml())